<a href="https://colab.research.google.com/github/mrdbourke/pytorch-deep-learning/blob/main/video_notebooks/00_pytorch_fundamentals_video.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## 00. PyTorch Fundamentals

Resource notebook: https://www.learnpytorch.io/00_pytorch_fundamentals/

If you have a question: https://github.com/mrdbourke/pytorch-deep-learning/discussions

In [2]:
import torch
print(torch.cuda.is_available())       # should print: True
print(torch.cuda.get_device_name(0))   # should print: NVIDIA GeForce RTX 5060


True
NVIDIA GeForce RTX 5060 Laptop GPU


In [3]:
import torch, time

# Confirm GPU
print(f"Device name: {torch.cuda.get_device_name(0)}")
print(f"CUDA available: {torch.cuda.is_available()}")
print(f"PyTorch version: {torch.__version__}")
print()

# --- 1. Simple matrix multiply benchmark ---
sizes = [512, 1024, 2048, 4096, 8192]
results = []

for n in sizes:
    A = torch.randn((n, n), device='cuda')
    B = torch.randn((n, n), device='cuda')

    torch.cuda.synchronize()
    start = time.time()
    C = torch.mm(A, B)
    torch.cuda.synchronize()
    end = time.time()

    gflops = (2 * n**3) / (end - start) / 1e9
    results.append((n, round(gflops, 2)))
    print(f"Size {n}x{n}: {gflops:.2f} GFLOPs")

# --- 2. Optional: simulate a small CNN training loop ---
import torch.nn as nn
import torch.optim as optim

class TinyCNN(nn.Module):
    def __init__(self):
        super().__init__()
        self.net = nn.Sequential(
            nn.Conv2d(3, 32, 3, padding=1), nn.ReLU(),
            nn.Conv2d(32, 64, 3, padding=1), nn.ReLU(),
            nn.Flatten(), nn.Linear(64*32*32, 10)
        )
    def forward(self, x): return self.net(x)

model = TinyCNN().cuda()
opt = optim.Adam(model.parameters())
criterion = nn.CrossEntropyLoss()

x = torch.randn(128, 3, 32, 32, device='cuda')
y = torch.randint(0, 10, (128,), device='cuda')

torch.cuda.synchronize()
t0 = time.time()
for _ in range(100):
    opt.zero_grad(set_to_none=True)
    loss = criterion(model(x), y)
    loss.backward()
    opt.step()
torch.cuda.synchronize()
t1 = time.time()

print(f"\n100 training steps took {t1 - t0:.2f} seconds")


Device name: NVIDIA GeForce RTX 5060 Laptop GPU
CUDA available: True
PyTorch version: 2.10.0.dev20251010+cu128

Size 512x512: 1.14 GFLOPs
Size 1024x1024: 1064.68 GFLOPs
Size 2048x2048: 11421.40 GFLOPs
Size 4096x4096: 8754.28 GFLOPs
Size 8192x8192: 9412.49 GFLOPs

100 training steps took 1.19 seconds


In [6]:
import torch
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
print(torch.__version__)

2.10.0.dev20251010+cu128


## Introduction to Tensors

### Creating tensors

PyTorch tensors are created using `torch.Tensor()` = https://pytorch.org/docs/stable/tensors.html

In [17]:
# scalar
scalar = torch.tensor(7)
scalar

tensor(7)

In [8]:
scalar.ndim

0

In [9]:
# Get tensor back as Python int
scalar.item()

7

In [10]:
# Vector
vector = torch.tensor([7, 7])
vector

tensor([7, 7])

In [11]:
vector.ndim

1

In [13]:
vector.shape

torch.Size([2])

In [15]:
# MATRIX
MATRIX = torch.tensor([[7, 8],
                       [9, 10]])
MATRIX

tensor([[ 7,  8],
        [ 9, 10]])

In [16]:
MATRIX.ndim

2

In [ ]:
MATRIX[1]

tensor([ 9, 10])

In [ ]:
MATRIX.shape

torch.Size([2, 2])

In [ ]:
# TENSOR
TENSOR = torch.tensor([[[1, 2, 3],
                        [3, 6, 9],
                        [2, 4, 5]]])

TENSOR

tensor([[[1, 2, 3],
         [3, 6, 9],
         [2, 4, 5]]])

In [ ]:
TENSOR.ndim

3

In [ ]:
TENSOR.shape

torch.Size([1, 3, 3])

In [ ]:
TENSOR[0]

tensor([[1, 2, 3],
        [3, 6, 9],
        [2, 4, 5]])

### Random tensors

Why random tensors?

Random tensors are important because the way many neural networks learn is that they start with tensors full of random numbers and then adjust those random numbers to better represent the data.

`Start with random numbers -> look at data -> update random numbers -> look at data -> update random numbers`

Torch random tensors - https://pytorch.org/docs/stable/generated/torch.rand.html

In [ ]:
# Create a random tensor of size (3, 4)
random_tensor = torch.rand(3, 4)
random_tensor

tensor([[0.4433, 0.7119, 0.4170, 0.4409],
        [0.8014, 0.2050, 0.3547, 0.6358],
        [0.3007, 0.1659, 0.3462, 0.7317]])

In [ ]:
# Create a random tensor with similar shape to an image tensor
random_image_size_tensor = torch.rand(size=(3, 224, 224)) # height, width, colour channels (R, G, B)
random_image_size_tensor.shape, random_image_size_tensor.ndim

(torch.Size([3, 224, 224]), 3)

### Zeros and ones

In [ ]:
# Create a tensor of all zeros
zeros = torch.zeros(size=(3, 4))
zeros

tensor([[0., 0., 0., 0.],
        [0., 0., 0., 0.],
        [0., 0., 0., 0.]])

In [ ]:
# Create a tensor of all ones
ones = torch.ones(size=(3, 4))
ones

tensor([[1., 1., 1., 1.],
        [1., 1., 1., 1.],
        [1., 1., 1., 1.]])

In [ ]:
ones.dtype

torch.float32

In [ ]:
random_tensor.dtype

torch.float32

### Creating a range of tensors and tensors-like 

In [ ]:
# Use torch.range() and get deprecated message, use torch.arange()
one_to_ten = torch.arange(start=1, end=11, step=1)
one_to_ten

tensor([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10])

In [ ]:
# Creating tensors like
ten_zeros = torch.zeros_like(input=one_to_ten)
ten_zeros

tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0])

### Tensor datatypes 

**Note:** Tensor datatypes is one of the 3 big errors you'll run into with PyTorch & deep learning:
1. Tensors not right datatype
2. Tensors not right shape
3. Tensors not on the right device

Precision in computing - https://en.wikipedia.org/wiki/Precision_(computer_science)#:~:text=In%20computer%20science%2C%20the%20precision,used%20to%20express%20a%20value.

In [ ]:
# Float 32 tensor
float_32_tensor = torch.tensor([3.0, 6.0, 9.0],
                               dtype=None, # what datatype is the tensor (e.g. float32 or float16)
                               device=None, # What device is your tensor on
                               requires_grad=False) # whether or not to track gradients with this tensors operations
float_32_tensor

tensor([3., 6., 9.])

In [ ]:
float_32_tensor.dtype

torch.float32

In [ ]:
float_16_tensor = float_32_tensor.type(torch.float16)
float_16_tensor

tensor([3., 6., 9.], dtype=torch.float16)

In [ ]:
float_16_tensor * float_32_tensor

tensor([ 9., 36., 81.])

In [ ]:
int_32_tensor = torch.tensor([3, 6, 9], dtype=torch.long)
int_32_tensor

tensor([3, 6, 9])

In [ ]:
float_32_tensor * int_32_tensor

tensor([ 9., 36., 81.])

### Getting information from tensors (tensor attributes)

1. Tensors not right datatype - to do get datatype from a tensor, can use `tensor.dtype`
2. Tensors not right shape - to get shape from a tensor, can use `tensor.shape`
3. Tensors not on the right device - to get device from a tensor, can use `tensor.device`

In [ ]:
# Create a tensor
some_tensor = torch.rand(3, 4)
some_tensor

tensor([[0.7151, 0.9288, 0.0464, 0.2910],
        [0.7281, 0.5272, 0.9098, 0.3145],
        [0.9641, 0.4652, 0.8553, 0.0232]])

In [ ]:
# Find out details about some tensor
print(some_tensor)
print(f"Datatype of tensor: {some_tensor.dtype}")
print(f"Shape of tensor: {some_tensor.shape}")
print(f"Device tensor is on: {some_tensor.device}")

tensor([[0.7151, 0.9288, 0.0464, 0.2910],
        [0.7281, 0.5272, 0.9098, 0.3145],
        [0.9641, 0.4652, 0.8553, 0.0232]])
Datatype of tensor: torch.float32
Shape of tensor: torch.Size([3, 4])
Device tensor is on: cpu


### Manipulating Tensors (tensor operations)

Tensor opertions include:
* Addition
* Subtraction
* Multiplication (element-wise)
* Division
* Matrix multiplication

In [ ]:
# Create a tensor and add 10 to it
tensor = torch.tensor([1, 2, 3])
tensor + 10

tensor([11, 12, 13])

In [ ]:
# Multiply tensor by 10
tensor * 10

tensor([10, 20, 30])

In [ ]:
tensor

tensor([1, 2, 3])

In [ ]:
# Substract 10
tensor - 10

tensor([-9, -8, -7])

In [ ]:
# Try out PyTorch in-built functions
torch.mul(tensor, 10)

tensor([10, 20, 30])

In [ ]:
torch.add(tensor, 10)

tensor([11, 12, 13])

### Matrix multiplication

Two main ways of performing multiplication in neural networks and deep learning:

1. Element-wise multiplication
2. Matrix mutliplication (dot product)

More information on multiplying matrices - https://www.mathsisfun.com/algebra/matrix-multiplying.html

There are two main rules that performing matrix mutliplication needs to satisfy:
1. The **inner dimensions** must match:
* `(3, 2) @ (3, 2)` won't work
* `(2, 3) @ (3, 2)` will work
* `(3, 2) @ (2, 3)` will work
2. The resulting matrix has the shape of the **outer dimensions**:
* `(2, 3) @ (3, 2)` -> `(2, 2)`
* `(3, 2) @ (2, 3)` -> `(3, 3)`

In [ ]:
# Element wise multiplication
print(tensor, "*", tensor)
print(f"Equals: {tensor * tensor}")

tensor([1, 2, 3]) * tensor([1, 2, 3])
Equals: tensor([1, 4, 9])


In [ ]:
# Matrix multiplication
torch.matmul(tensor, tensor)

tensor(14)

In [ ]:
tensor

tensor([1, 2, 3])

In [ ]:
# Matrix multiplication by hand
1*1 + 2*2 + 3*3

14

In [ ]:
%%time
value = 0
for i in range(len(tensor)):
  value += tensor[i] * tensor[i]
print(value)

tensor(14)
CPU times: user 661 µs, sys: 876 µs, total: 1.54 ms
Wall time: 1.55 ms


In [ ]:
%%time
torch.matmul(tensor, tensor)

CPU times: user 67 µs, sys: 29 µs, total: 96 µs
Wall time: 101 µs


tensor(14)

### One of the most common errors in deep learning: shape errors

In [ ]:
# Shapes for matrix multiplication 
tensor_A = torch.tensor([[1, 2],
                         [3, 4],
                         [5, 6]])

tensor_B = torch.tensor([[7, 10],
                         [8, 11],
                         [9, 12]])

# torch.mm(tensor_A, tensor_B) # torch.mm is the same as torch.matmul (it's an alias for writing less code)
torch.matmul(tensor_A, tensor_B)

RuntimeError: ignored

In [ ]:
tensor_B.T

tensor([[ 7,  8,  9],
        [10, 11, 12]])

In [ ]:
tensor_A.shape, tensor_B.shape

(torch.Size([3, 2]), torch.Size([3, 2]))

To fix our tensor shape issues, we can manipulate the shape of one of our tensors using a **transpose**.

A **transpose** switches the axes or dimensions of a given tensor.

In [ ]:
tensor_B, tensor_B.shape

(tensor([[ 7, 10],
         [ 8, 11],
         [ 9, 12]]), torch.Size([3, 2]))

In [ ]:
tensor_B.T, tensor_B.T.shape

(tensor([[ 7,  8,  9],
         [10, 11, 12]]), torch.Size([2, 3]))

In [ ]:
# The matrix multiplication operation works when tensor_B is transposed
print(f"Original shapes: tensor_A = {tensor_A.shape}, tensor_B = {tensor_B.shape}")
print(f"New shapes: tensor_A = {tensor_A.shape} (same shape as above), tensor_B.T = {tensor_B.T.shape}")
print(f"Multiplying: {tensor_A.shape} @ {tensor_B.T.shape} <- inner dimensions must match")
print("Output:\n")
output = torch.matmul(tensor_A, tensor_B.T)
print(output) 
print(f"\nOutput shape: {output.shape}")

Original shapes: tensor_A = torch.Size([3, 2]), tensor_B = torch.Size([3, 2])
New shapes: tensor_A = torch.Size([3, 2]) (same shape as above), tensor_B.T = torch.Size([2, 3])
Multiplying: torch.Size([3, 2]) @ torch.Size([2, 3]) <- inner dimensions must match
Output:

tensor([[ 27,  30,  33],
        [ 61,  68,  75],
        [ 95, 106, 117]])

Output shape: torch.Size([3, 3])


## Finding the min, max, mean, sum, etc (tensor aggregation)

In [ ]:
# Create a tensor
x = torch.arange(1, 100, 10)
x, x.dtype

(tensor([ 1, 11, 21, 31, 41, 51, 61, 71, 81, 91]), torch.int64)

In [ ]:
# Find the min
torch.min(x), x.min()

(tensor(1), tensor(1))

In [ ]:
# Find the max
torch.max(x), x.max()

(tensor(91), tensor(91))

In [ ]:
# Find the mean - note: the torch.mean() function requires a tensor of float32 datatype to work
torch.mean(x.type(torch.float32)), x.type(torch.float32).mean()

(tensor(46.), tensor(46.))

In [ ]:
# Find the sum
torch.sum(x), x.sum()

(tensor(460), tensor(460))

## Finding the positional min and max

In [ ]:
x

tensor([ 1, 11, 21, 31, 41, 51, 61, 71, 81, 91])

In [ ]:
# Find the position in tensor that has the minimum value with argmin() -> returns index position of targt tensor where the minimum value occurs 
x.argmin()

tensor(0)

In [ ]:
x[0]

tensor(1)

In [ ]:
# Find the position in tensor that has the maximum value with argmax()
x.argmax()

tensor(9)

In [ ]:
x[9]

tensor(91)

## Reshaping, stacking, squeezing and unsqueezing tensors

* Reshaping - reshapes an input tensor to a defined shape
* View - Return a view of an input tensor of certain shape but keep the same memory as the original tensor
* Stacking - combine multiple tensors on top of each other (vstack) or side by side (hstack)
* Squeeze - removes all `1` dimensions from a tensor
* Unsqueeze - add a `1` dimension to a target tensor
* Permute - Return a view of the input with dimensions permuted (swapped) in a certain way

In [ ]:
# Let's create a tensor
import torch
x = torch.arange(1., 10.)
x, x.shape

(tensor([1., 2., 3., 4., 5., 6., 7., 8., 9.]), torch.Size([9]))

In [ ]:
# Add an extra dimension
x_reshaped = x.reshape(1, 9)
x_reshaped, x_reshaped.shape

(tensor([[1., 2., 3., 4., 5., 6., 7., 8., 9.]]), torch.Size([1, 9]))

In [ ]:
# Change the view 
z = x.view(1, 9)
z, z.shape

(tensor([[1., 2., 3., 4., 5., 6., 7., 8., 9.]]), torch.Size([1, 9]))

In [ ]:
# Changing z changes x (because a view of a tensor shares the same memory as the original input)
z[:, 0] = 5
z, x

(tensor([[5., 2., 3., 4., 5., 6., 7., 8., 9.]]),
 tensor([5., 2., 3., 4., 5., 6., 7., 8., 9.]))

In [ ]:
# Stack tensors on top of each other
x_stacked = torch.stack([x, x, x, x], dim=0)
x_stacked

tensor([[5., 2., 3., 4., 5., 6., 7., 8., 9.],
        [5., 2., 3., 4., 5., 6., 7., 8., 9.],
        [5., 2., 3., 4., 5., 6., 7., 8., 9.],
        [5., 2., 3., 4., 5., 6., 7., 8., 9.]])

In [ ]:
# torch.squeeze() - removes all single dimensions from a target tensor
print(f"Previous tensor: {x_reshaped}")
print(f"Previous shape: {x_reshaped.shape}")

# Remove extra dimensions from x_reshaped
x_squeezed = x_reshaped.squeeze()
print(f"\nNew tensor: {x_squeezed}")
print(f"New shape: {x_squeezed.shape}")

Previous tensor: tensor([[5., 2., 3., 4., 5., 6., 7., 8., 9.]])
Previous shape: torch.Size([1, 9])

New tensor: tensor([5., 2., 3., 4., 5., 6., 7., 8., 9.])
New shape: torch.Size([9])


In [ ]:
# torch.unsqueeze() - adds a single dimension to a target tensor at a specific dim (dimension)
print(f"Previous target: {x_squeezed}")
print(f"Previous shape: {x_squeezed.shape}")

# Add an extra dimension with unsqueeze
x_unsqueezed = x_squeezed.unsqueeze(dim=0)
print(f"\nNew tensor: {x_unsqueezed}")
print(f"New shape: {x_unsqueezed.shape}")

Previous target: tensor([5., 2., 3., 4., 5., 6., 7., 8., 9.])
Previous shape: torch.Size([9])

New tensor: tensor([[5., 2., 3., 4., 5., 6., 7., 8., 9.]])
New shape: torch.Size([1, 9])


In [ ]:
# torch.permute - rearranges the dimensions of a target tensor in a specified order
x_original = torch.rand(size=(224, 224, 3)) # [height, width, colour_channels]

# Permute the original tensor to rearrange the axis (or dim) order
x_permuted = x_original.permute(2, 0, 1) # shifts axis 0->1, 1->2, 2->0

print(f"Previous shape: {x_original.shape}") 
print(f"New shape: {x_permuted.shape}") # [colour_channels, height, width]

Previous shape: torch.Size([224, 224, 3])
New shape: torch.Size([3, 224, 224])


In [ ]:
x_original[0, 0, 0] = 728218
x_original[0, 0, 0], x_permuted[0, 0, 0]

(tensor(728218.), tensor(728218.))

## Indexing (selecting data from tensors)

Indexing with PyTorch is similar to indexing with NumPy.

In [ ]:
# Create a tensor
import torch
x = torch.arange(1, 10).reshape(1, 3, 3)
x, x.shape

(tensor([[[1, 2, 3],
          [4, 5, 6],
          [7, 8, 9]]]), torch.Size([1, 3, 3]))

In [ ]:
# Let's index on our new tensor
x[0]

tensor([[1, 2, 3],
        [4, 5, 6],
        [7, 8, 9]])

In [ ]:
# Let's index on the middle bracket (dim=1)
x[0][0]

tensor([1, 2, 3])

In [ ]:
# Let's index on the most inner bracket (last dimension)
x[0][1][1]

tensor(5)

In [ ]:
# You can also use ":" to select "all" of a target dimension
x[:, 0]

tensor([[1, 2, 3]])

In [ ]:
# Get all values of 0th and 1st dimensions but only index 1 of 2nd dimension
x[:, :, 1]

tensor([[2, 5, 8]])

In [ ]:
# Get all values of the 0 dimension but only the 1 index value of 1st and 2nd dimension
x[:, 1, 1]

tensor([5])

In [ ]:
# Get index 0 of 0th and 1st dimension and all values of 2nd dimension
x[0, 0, :]

tensor([1, 2, 3])

In [ ]:
# Index on x to return 9
print(x[0][2][2])

# Index on x to return 3, 6, 9
print(x[:, :, 2])

tensor(9)
tensor([[3, 6, 9]])


## PyTorch tensors & NumPy

NumPy is a popular scientific Python numerical computing library. 

And because of this, PyTorch has functionality to interact with it.

* Data in NumPy, want in PyTorch tensor -> `torch.from_numpy(ndarray)`
* PyTorch tensor -> NumPy -> `torch.Tensor.numpy()` 

In [ ]:
# NumPy array to tensor
import torch
import numpy as np

array = np.arange(1.0, 8.0)
tensor = torch.from_numpy(array) # warning: when converting from numpy -> pytorch, pytorch reflects numpy's default datatype of float64 unless specified otherwise
array, tensor

(array([1., 2., 3., 4., 5., 6., 7.]),
 tensor([1., 2., 3., 4., 5., 6., 7.], dtype=torch.float64))

In [ ]:
# Change the value of array, what will this do to `tensor`?
array = array + 1
array, tensor

(array([2., 3., 4., 5., 6., 7., 8.]),
 tensor([1., 2., 3., 4., 5., 6., 7.], dtype=torch.float64))

In [ ]:
# Tensor to NumPy array
tensor = torch.ones(7)
numpy_tensor = tensor.numpy()
tensor, numpy_tensor

(tensor([1., 1., 1., 1., 1., 1., 1.]),
 array([1., 1., 1., 1., 1., 1., 1.], dtype=float32))

In [ ]:
# Change the tesnor, what happens to `numpy_tensor`?
tensor = tensor + 1
tensor, numpy_tensor 

(tensor([2., 2., 2., 2., 2., 2., 2.]),
 array([1., 1., 1., 1., 1., 1., 1.], dtype=float32))

## Reproducbility (trying to take random out of random)

In short how a neural network learns:

`start with random numbers -> tensor operations -> update random numbers to try and make them better representations of the data -> again -> again -> again...`

To reduce the randomness in neural networks and PyTorch comes the concept of a **random seed**.

Essentially what the random seed does is "flavour" the randomness.

In [ ]:
import torch

# Create two random tensors
random_tensor_A = torch.rand(3, 4)
random_tensor_B = torch.rand(3, 4)

print(random_tensor_A)
print(random_tensor_B)
print(random_tensor_A == random_tensor_B)

tensor([[0.3675, 0.8410, 0.0507, 0.3165],
        [0.7275, 0.9676, 0.3901, 0.8840],
        [0.5177, 0.2239, 0.4362, 0.3602]])
tensor([[0.5229, 0.6719, 0.2790, 0.8198],
        [0.6689, 0.8659, 0.7849, 0.4268],
        [0.2076, 0.8076, 0.4377, 0.2555]])
tensor([[False, False, False, False],
        [False, False, False, False],
        [False, False, False, False]])


In [ ]:
# Let's make some random but reproducible tensors
import torch

# Set the random seed
RANDOM_SEED = 42
torch.manual_seed(RANDOM_SEED)
random_tensor_C = torch.rand(3, 4)

torch.manual_seed(RANDOM_SEED)
random_tensor_D = torch.rand(3, 4)

print(random_tensor_C)
print(random_tensor_D)
print(random_tensor_C == random_tensor_D)

tensor([[0.8823, 0.9150, 0.3829, 0.9593],
        [0.3904, 0.6009, 0.2566, 0.7936],
        [0.9408, 0.1332, 0.9346, 0.5936]])
tensor([[0.8823, 0.9150, 0.3829, 0.9593],
        [0.3904, 0.6009, 0.2566, 0.7936],
        [0.9408, 0.1332, 0.9346, 0.5936]])
tensor([[True, True, True, True],
        [True, True, True, True],
        [True, True, True, True]])


Extra resources for reproducibility:
* https://pytorch.org/docs/stable/notes/randomness.html
* https://en.wikipedia.org/wiki/Random_seed

 ## Running tensors and PyTorch objects on the GPUs (and making faster computations)

 GPUs = faster computation on numbers, thanks to CUDA + NVIDIA hardware + PyTorch working behind the scenes to make everything hunky dory (good).

### 1. Getting a GPU

1. Easiest - Use Google Colab for a free GPU (options to upgrade as well)
2. Use your own GPU - takes a little bit of setup and requires the investment of purchasing a GPU, there's lots of options..., see this post for what option to get: https://timdettmers.com/2020/09/07/which-gpu-for-deep-learning/
3. Use cloud computing - GCP, AWS, Azure, these services allow you to rent computers on the cloud and access them

For 2, 3 PyTorch + GPU drivers (CUDA) takes a little bit of setting up, to do this, refer to PyTorch setup documentation: https://pytorch.org/get-started/locally/ 

In [ ]:
!nvidia-smi

Sun Feb 20 00:24:35 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   32C    P0    28W / 250W |      0MiB / 16280MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

### 2. Check for GPU access with PyTorch

In [ ]:
# Check for GPU access with PyTorch
import torch
torch.cuda.is_available()

True

For PyTorch since it's capable of running compute on the GPU or CPU, it's best practice to setup device agnostic code: https://pytorch.org/docs/stable/notes/cuda.html#best-practices

E.g. run on GPU if available, else default to CPU

In [ ]:
# Setup device agnostic code 
device = "cuda" if torch.cuda.is_available() else "cpu"
device

'cuda'

In [ ]:
# Count number of devices
torch.cuda.device_count()

1

## 3. Putting tensors (and models) on the GPU

The reason we want our tensors/models on the GPU is because using a GPU results in faster computations.

In [ ]:
# Create a tensor (default on the CPU)
tensor = torch.tensor([1, 2, 3])

# Tensor not on GPU
print(tensor, tensor.device)

tensor([1, 2, 3]) cpu


In [ ]:
# Move tensor to GPU (if available)
tensor_on_gpu = tensor.to(device)
tensor_on_gpu

tensor([1, 2, 3], device='cuda:0')

### 4. Moving tensors back to the CPU

In [ ]:
# If tensor is on GPU, can't transform it to NumPy
tensor_on_gpu.numpy()

TypeError: ignored

In [ ]:
# To fix the GPU tensor with NumPy issue, we can first set it to the CPU
tensor_back_on_cpu = tensor_on_gpu.cpu().numpy()
tensor_back_on_cpu

array([1, 2, 3])

In [ ]:
tensor_on_gpu

tensor([1, 2, 3], device='cuda:0')

## Exercises & Extra-curriculum

See exercises for this notebook here: https://www.learnpytorch.io/00_pytorch_fundamentals/#exercises 
See the template exercises notebook for this module here: https://github.com/mrdbourke/pytorch-deep-learning/blob/main/extras/exercises/00_pytorch_fundamentals_exercises.ipynb 

In [33]:
import cupy as cp
print(cp.show_config())



OS                           : Windows-10-10.0.26100-SP0
Python Version               : 3.10.18
CuPy Version                 : 13.6.0
CuPy Platform                : NVIDIA CUDA
NumPy Version                : 2.0.1
SciPy Version                : 1.15.3
Cython Build Version         : 3.0.12
Cython Runtime Version       : None
CUDA Root                    : None
nvcc PATH                    : None
CUDA Build Version           : 12090
CUDA Driver Version          : 12080
CUDA Runtime Version         : 12090 (linked to CuPy) / 12080 (locally installed)
CUDA Extra Include Dirs      : []
cuBLAS Version               : (available)
cuFFT Version                : 11303
cuRAND Version               : 10309
cuSOLVER Version             : (11, 7, 3)
cuSPARSE Version             : (available)
NVRTC Version                : (12, 8)
Thrust Version               : 200800
CUB Build Version            : 200800
Jitify Build Version         : 1a0ca0e
cuDNN Build Version          : (not loaded; try `import 

In [ ]:
# ==========================================================
# Dijkstra GPU-Only Implementation (CuPy)
# ==========================================================

import random
import time
import cupy as cp
import matplotlib.pyplot as plt

# -------------------------------
# Input Generators
# -------------------------------
def generateInputMatrix_v(v, prob):
    edge = 0
    matrix = [[float('inf') for _ in range(v)] for _ in range(v)]
    for i in range(v):
        for j in range(v):
            if i == j:
                matrix[i][j] = 0
            elif matrix[i][j] == float('inf'):
                if random.random() < prob:
                    matrix[i][j] = random.randint(1, 50)
                    edge += 1
    return edge, matrix


# -------------------------------
# GPU: Dijkstra (Matrix-based with CuPy)
# -------------------------------
def dijkstra_gpu(adjacency_matrix, start):
    """
    GPU-accelerated Dijkstra using CuPy.
    Works efficiently for dense graphs represented as adjacency matrices.
    """
    adj = cp.asarray(adjacency_matrix, dtype=cp.float32)
    n = adj.shape[0]

    visited = cp.zeros(n, dtype=cp.bool_)
    dist = cp.full(n, cp.inf, dtype=cp.float32)
    dist[start] = 0

    for _ in range(n):
        # Find nearest unvisited vertex (parallel argmin)
        mask = cp.where(visited, cp.inf, dist)
        u = int(cp.argmin(mask))
        visited[u] = True

        # Update all neighbors in parallel
        new_dists = dist[u] + adj[u]
        dist = cp.minimum(dist, new_dists)

    cp.cuda.Stream.null.synchronize()  # wait for GPU
    return cp.asnumpy(dist)


# ==========================================================
# Run GPU Dijkstra + Plot Results
# ==========================================================

v_values = [100, 1000, 2000, 5000, 10000, 20000]   # Adjust as needed
gpu_times = []
edge_values = []
density = 1.0

print("Running GPU-accelerated Dijkstra...")

for v in v_values:
    edges, matrix = generateInputMatrix_v(v, density)
    edge_values.append(edges)

    start_time = time.perf_counter()
    d_gpu = dijkstra_gpu(matrix, 0)
    cp.cuda.Stream.null.synchronize()
    elapsed = time.perf_counter() - start_time
    gpu_times.append(elapsed)

    print(f"V={v:<6d} | Edges={edges:<10d} | GPU Time={elapsed:.3f}s")

# ==========================================================
# Plot Results
# ==========================================================
plt.figure(figsize=(7,5))
plt.plot(v_values, gpu_times, 'o-', label='GPU (CuPy)')
plt.xlabel("# of Vertices (V)")
plt.ylabel("Time (s)")
plt.title(f"Dijkstra GPU Performance (Density={density})")
plt.legend()
plt.grid(True)
plt.show()

plt.figure(figsize=(7,5))
plt.plot(edge_values, gpu_times, 's--', label='GPU (CuPy)')
plt.xlabel("# of Edges (E)")
plt.xscale('log')
plt.ylabel("Time (s)")
plt.title(f"Dijkstra GPU Performance vs Edges (Density={density})")
plt.legend()
plt.grid(True)
plt.show()


In [24]:
!pip install cupy-cuda12x


In [22]:
!pip install cupy-cuda12x


^C
   ---------------------------------------- 0.0/89.9 MB ? eta -:--:--
   - -------------------------------------- 2.4/89.9 MB 11.2 MB/s eta 0:00:08
   -- ------------------------------------- 4.7/89.9 MB 11.4 MB/s eta 0:00:08
   --- ------------------------------------ 6.8/89.9 MB 10.8 MB/s eta 0:00:08
   --- ------------------------------------ 8.9/89.9 MB 10.7 MB/s eta 0:00:08
   ---- ----------------------------------- 10.5/89.9 MB 10.4 MB/s eta 0:00:08
   ---- ----------------------------------- 10.5/89.9 MB 10.4 MB/s eta 0:00:08
   ---- ----------------------------------- 10.5/89.9 MB 10.4 MB/s eta 0:00:08
   ---- ----------------------------------- 11.0/89.9 MB 6.5 MB/s eta 0:00:13
   ----- ---------------------------------- 12.8/89.9 MB 6.8 MB/s eta 0:00:12
   ------ --------------------------------- 14.9/89.9 MB 7.0 MB/s eta 0:00:11
   ------- -------------------------------- 17.0/89.9 MB 7.3 MB/s eta 0:00:11
   -------- ------------------------------- 19.1/89.9 MB 7.5 MB/s 

In [26]:
!pip install -U --pre cupy-cuda12x


In [30]:
import os
os.environ["CUPY_ACCELERATORS"] = "nvrtc"
import cupy as cp
print("✅ CuPy version:", cp.__version__)
print("✅ GPU detected:", cp.cuda.runtime.getDeviceProperties(0)['name'])

✅ CuPy version: 13.6.0
✅ GPU detected: b'NVIDIA GeForce RTX 5060 Laptop GPU'


In [32]:
import os
os.environ["CUDA_PATH"] = ""
os.environ["CUPY_DUMMY_CUDA_PATH"] = "1"

import heapq
import random
import time
import matplotlib.pyplot as plt

# -------------------------------
# CPU: Dijkstra (Matrix-based)
# -------------------------------
def dijkstra1(adjacency_matrix, start):
    n = len(adjacency_matrix)
    V = [i for i in range(n)]
    d = {node: float('inf') for node in V}
    pi = {node: None for node in V}
    visited = set()
    d[start] = 0

    while len(visited) < n:
        # Find nearest unvisited node
        unvisited = [i for i in range(n) if i not in visited]
        nearest = min(unvisited, key=lambda node: d[node])

        visited.add(nearest)
        for i in range(n):
            w = adjacency_matrix[nearest][i]
            if w != 0 and w != float('inf') and i not in visited:
                if d[nearest] + w < d[i]:
                    d[i] = d[nearest] + w
                    pi[i] = nearest

    return d, pi


# -------------------------------
# CPU: Dijkstra (Adjacency List + MinHeap)
# -------------------------------
def dijk_minheap(adjacency_list, start):
    n = len(adjacency_list)
    d = {node: float('inf') for node in range(n)}
    pi = {node: None for node in range(n)}
    d[start] = 0
    visited = set()

    q = [(0, start)]
    
    while len(visited) < n:
        min_length, nearest = heapq.heappop(q)
        visited.add(nearest)
        
        if min_length > d[nearest]:
            continue
        else:
            for i, w in adjacency_list[nearest]:
                if d[nearest] + w < d[i]:
                    d[i] = d[nearest] + w
                    pi[i] = nearest
                    heapq.heappush(q, (d[i], i))

    return d, pi


# -------------------------------
# Input Generators
# -------------------------------
def generateInputMatrix_v(v, prob):
    edge = 0
    matrix = [[float('inf') for _ in range(v)] for _ in range(v)]
    for i in range(v):
        for j in range(v):
            if i == j:
                matrix[i][j] = 0
            elif matrix[i][j] == float('inf'):
                if random.random() < prob:
                    matrix[i][j] = random.randint(1, 50)
                    edge += 1
    return edge, matrix


def generateInputArray(v, prob):
    edge = 0
    adj_lists = [[] for _ in range(v)]
    for i in range(v):
        for j in range(v):
            if i == j:
                continue
            elif random.random() < prob:
                node = j
                adj_lists[i].append((node, random.randint(1, 50)))
                edge += 1
    return edge, adj_lists


def convertMatrixToList(matrix):
    adj_lists = [[] for _ in range(len(matrix))]
    for i in range(len(matrix)):
        for j in range(len(matrix)):
            if matrix[i][j] != 0 and matrix[i][j] != float('inf'):
                adj_lists[i].append((j, matrix[i][j]))
    return adj_lists


# -------------------------------
# GPU: Dijkstra (Matrix-based with CuPy)
# -------------------------------
import cupy as cp

def dijkstra_gpu(adjacency_matrix, start):
    """
    GPU-accelerated Dijkstra using CuPy.
    Works efficiently for dense graphs represented as adjacency matrices.
    """
    adj = cp.asarray(adjacency_matrix, dtype=cp.float32)
    n = adj.shape[0]

    visited = cp.zeros(n, dtype=cp.bool_)
    dist = cp.full(n, cp.inf, dtype=cp.float32)
    dist[start] = 0

    for _ in range(n):
        # Find nearest unvisited vertex (parallel argmin)
        mask = cp.where(visited, cp.inf, dist)
        u = int(cp.argmin(mask))
        visited[u] = True

        # Update all neighbors in parallel
        new_dists = dist[u] + adj[u]
        dist = cp.minimum(dist, new_dists)

    cp.cuda.Stream.null.synchronize()  # wait for GPU
    return cp.asnumpy(dist)


# -------------------------------
# Benchmark: CPU vs GPU (Matrix version)
# -------------------------------
v_values = [100, 1000, 2000, 5000]
cpu_times = []
gpu_times = []

density = 1.0
print("Running CPU vs GPU Dijkstra comparison...")

for v in v_values:
    edges, matrix = generateInputMatrix_v(v, density)

    # CPU
    start = time.perf_counter()
    d_cpu, _ = dijkstra1(matrix, 0)
    cpu_times.append(time.perf_counter() - start)

    # GPU
    start = time.perf_counter()
    d_gpu = dijkstra_gpu(matrix, 0)
    cp.cuda.Stream.null.synchronize()
    gpu_times.append(time.perf_counter() - start)

    print(f"V={v:<6d} | CPU={cpu_times[-1]:.3f}s | GPU={gpu_times[-1]:.3f}s")

# Plot comparison
plt.figure(figsize=(7,5))
plt.plot(v_values, cpu_times, 'o-', label='CPU')
plt.plot(v_values, gpu_times, 's--', label='GPU')
plt.xlabel('# of Vertices (V)')
plt.ylabel('Execution Time (s)')
plt.title('Dijkstra CPU vs GPU (Dense Graphs)')
plt.legend()
plt.grid(True)
plt.show()


# -------------------------------
# Benchmark: CPU (Adjacency List version)
# -------------------------------
v_values = [100, 1000, 2000, 5000]
cpu_times = []
edge_values = []

for v in v_values:
    edges, lists = generateInputArray(v, density)
    edge_values.append(edges)
    start_time = time.perf_counter()
    d, pi = dijk_minheap(lists, 0)
    end_time = time.perf_counter()
    cpu_times.append(end_time - start_time)
    print(f"AdjList CPU Dijkstra | V={v:<6d} | Time={cpu_times[-1]:.3f}s")

# Plot
plt.figure(figsize=(7,5))
plt.plot(v_values, cpu_times, marker="o")
plt.xlabel("# of Vertices (V)")
plt.ylabel("CPU Time (s)")
plt.title(f"Dijkstra (Adj List, Density = {density})")
plt.grid(True)
plt.show()


Running CPU vs GPU Dijkstra comparison...


RuntimeError: Failed to auto-detect CUDA root directory. Please specify `CUDA_PATH` environment variable if you are using CUDA versions not yet supported by CuPy.